* https://explorer.rollux.com/address/0x8A4AA176007196D48d39C89402d3753c39AE64c1
* https://pypi.org/project/Historic-Crypto/
* https://github.com/man-c/pycoingecko/tree/master

In [14]:
from blockscout import Blockscout
from blockscout import Net
from blockscout import API
from datetime import datetime
import pandas as pd
import numpy as np
import time
from copy import deepcopy
import matplotlib.pyplot as plt

sys_addr = '0x6daF055c99883D920849D7022f2EfABb13e2af57'
psys_addr = '0x6daF055c99883D920849D7022f2EfABb13e2af57'
usdc_addr = '0x368433CaC2A0B8D76E64681a9835502a1f2A8A30'

whale_addr1 = '0x455829781E218d33D3587D8282692C8F09d76f1C'
whale_addr = '0x8A4AA176007196D48d39C89402d3753c39AE64c1'

tkn_addr = whale_addr1

eth_rpc = Blockscout(Net.ROLLUX, API.RPC)  
eth_rest = Blockscout(Net.ROLLUX, API.REST) 
tkn_addr

'0x455829781E218d33D3587D8282692C8F09d76f1C'

In [15]:
erc20_tkn_balances = eth_rpc.get_erc20_tokens_by_address(address=tkn_addr)

url: https://explorer.rollux.com/api?module=account&action=tokenlist&address=0x455829781E218d33D3587D8282692C8F09d76f1C


In [16]:
tkn_transfers = []
page_nm = 1
pull_tkn_transfers = True
while(pull_tkn_transfers):
    erc20_tkn_transfers = eth_rpc.get_erc20_token_transfer_events_by_address(address=tkn_addr, page=page_nm, offset=0, sort="desc")
    tkn_transfers.extend(erc20_tkn_transfers['result'])
    pull_tkn_transfers = pull_tkn_transfers if len(erc20_tkn_transfers['result']) > 0 else False
    page_nm+=1

url: https://explorer.rollux.com/api?module=account&action=tokentx&address=0x455829781E218d33D3587D8282692C8F09d76f1C&page=1&offset=0&sort=desc
url: https://explorer.rollux.com/api?module=account&action=tokentx&address=0x455829781E218d33D3587D8282692C8F09d76f1C&page=2&offset=0&sort=desc


In [24]:
df_transfers = pd.DataFrame(columns=['blk_num', 'timestamp', 'tkn_symbol', 'tkn_name', 'tkn_decimal', 
                                     'transfer_value', 'coin_value', 'tkn_balance', 'transfer_gas', 'transfer_in', 
                                     'tkn_address', 'hash'])

n_transfers = len(tkn_transfers)
c = 0
for k in range(n_transfers):
    tx = tkn_transfers[k]
    if('value' in tx):
        blk_num = int(tx['blockNumber'])
        timestamp = int(tx['timeStamp'])
        tkn_symbol = tx['tokenSymbol']
        tkn_name = tx['tokenName']
        tkn_decimal = int(tx['tokenDecimal'])
        tkn_address = tx['contractAddress']
        transfer_value = int(tx['value'])
        coin_value = transfer_value/(10**tkn_decimal)
        transfer_in = tx['to'] == tkn_addr.lower()
        transfer_gas = int(tx['gasUsed'])
        transfer_hash = tx['hash']
        df_transfers.loc[c] = [blk_num, timestamp, tkn_symbol, tkn_name, tkn_decimal, transfer_value, coin_value, 0, transfer_gas, transfer_in, tkn_address, transfer_hash]
        c+=1

df_transfers.sort_values(by=['blk_num'], ascending=False, inplace=False)
dict_transfers = df_transfers.to_dict('index')

In [9]:
dict_transfers = {}

n_transfers = len(tkn_transfers)
for k in range(n_transfers):
    tx = tkn_transfers[k]
    dict_transfers[k] = {}
    dict_transfers[k]['blk_num'] = int(tx['blockNumber'])
    dict_transfers[k]['timestamp'] = int(tx['timeStamp'])
    dict_transfers[k]['tkn_symbol'] = tx['tokenSymbol']
    dict_transfers[k]['tkn_name'] = tx['tokenName']
    dict_transfers[k]['tkn_decimal'] = int(tx['tokenDecimal'])
    dict_transfers[k]['tkn_address'] = tx['contractAddress']
    dict_transfers[k]['transfer_value'] = int(tx['value'])
    transfer_value = dict_transfers[k]['transfer_value']
    tkn_decimal = dict_transfers[k]['tkn_decimal']
    dict_transfers[k]['coin_value'] = transfer_value/(10**tkn_decimal)
    dict_transfers[k]['transfer_in'] = tx['to'] == tkn_addr.lower()
    dict_transfers[k]['transfer_gas'] = int(tx['gasUsed'])
    dict_transfers[k]['transfer_hash'] = tx['hash']

In [10]:
dict_transfers = dict(sorted(dict_transfers.items(), key=lambda item: item[1]['blk_num'], reverse=True))

dict_transfers_reindex = {}
for k, old_k in enumerate(dict_transfers):
    dict_transfers_reindex[k] = dict_transfers[old_k]
dict_transfers = dict_transfers_reindex

In [11]:
df_transfers = pd.DataFrame.from_dict(dict_transfers_reindex, orient='index')

In [12]:
init_tkn_balances = {}
for tkn in erc20_tkn_balances['result']:
    symbol = tkn['symbol']
    init_tkn_balances[symbol] = {} if symbol not in init_tkn_balances else {}
    init_tkn_balances[symbol]['tkn_balance'] = int(tkn['balance'])
    init_tkn_balances[symbol]['tkn_name'] = tkn['name']
    init_tkn_balances[symbol]['tkn_decimal'] = int(tkn['decimals'])
init_tkn_balances

{'USDT': {'tkn_balance': 89590009236, 'tkn_name': 'Tether', 'tkn_decimal': 6},
 'BTC': {'tkn_balance': 999799972, 'tkn_name': 'Bitcoin', 'tkn_decimal': 8},
 'USDC': {'tkn_balance': 238003477125,
  'tkn_name': 'USD Coin',
  'tkn_decimal': 6},
 'WSYS': {'tkn_balance': 65787771636449164370510,
  'tkn_name': 'Wrapped Syscoin',
  'tkn_decimal': 18},
 'ETH': {'tkn_balance': 2689089808644384766235,
  'tkn_name': 'Ethereum',
  'tkn_decimal': 18}}

In [13]:
tkn_balances = {}
N = len(dict_transfers)
for k in range(N, 0, -1):
    tx = dict_transfers[k-1]
    tkn_symbol = tx['tkn_symbol']
    transfer_value = tx['transfer_value']
    transfer_in = tx['transfer_in']

    tkn_balances[tkn_symbol] = {} if tkn_symbol not in tkn_balances else tkn_balances[tkn_symbol]
    tkn_balances[tkn_symbol]['tkn_balance'] = 0 if 'tkn_balance' not in tkn_balances[tkn_symbol] else tkn_balances[tkn_symbol]['tkn_balance']

    if(transfer_in):
        tkn_balances[tkn_symbol]['tkn_balance'] += transfer_value
    elif tkn_balances[tkn_symbol]['tkn_balance'] >= transfer_value:     
        tkn_balances[tkn_symbol]['tkn_balance'] -= transfer_value
    else:
        tkn_balances[tkn_symbol]['tkn_balance'] = 0

    dict_transfers[k-1]['tkn_balance'] = tkn_balances[tkn_symbol]['tkn_balance']

tkn_balances

{'USDT': {'tkn_balance': 89590009236},
 'USDC': {'tkn_balance': 238003477125},
 'ETH': {'tkn_balance': 2689089808644384766235},
 'BTC': {'tkn_balance': 999799972},
 'WSYS': {'tkn_balance': 65787771636449164370510}}

In [ ]:
tkn_balances = {}
N = len(df_transfers)
for k in range(N, 0, -1):
    tx = df_transfers.iloc[k-1]
    tkn_symbol = tx['tkn_symbol']
    transfer_value = tx['transfer_value']
    transfer_in = tx['transfer_in']

    tkn_balances[tkn_symbol] = {} if tkn_symbol not in tkn_balances else tkn_balances[tkn_symbol]
    tkn_balances[tkn_symbol]['tkn_balance'] = 0 if 'tkn_balance' not in tkn_balances[tkn_symbol] else tkn_balances[tkn_symbol]['tkn_balance']

    if(transfer_in):
        tkn_balances[tkn_symbol]['tkn_balance'] += transfer_value
    elif tkn_balances[tkn_symbol]['tkn_balance'] >= transfer_value:     
        tkn_balances[tkn_symbol]['tkn_balance'] -= transfer_value
    else:
        tkn_balances[tkn_symbol]['tkn_balance'] = 0

    df_transfers.loc[k-1,'tkn_balance'] = tkn_balances[tkn_symbol]['tkn_balance']

tkn_balances

In [ ]:
df_transfers.head(4)

In [ ]:
dict_transfers[0]

In [ ]:
tkn_symbol = 'USDC'
plot_title = True

for tkn_symbol in tkn_balances.keys():
    tkn_decimal = init_tkn_balances[tkn_symbol]['tkn_decimal']
    df_tkn = df_transfers.loc[df_transfers['tkn_symbol'] == tkn_symbol].copy()
    df_tkn.sort_values(by=['blk_num'], ascending=True, inplace=True)
    df_tkn.reset_index(drop=True, inplace=True)
    
    dates = np.array([datetime.fromtimestamp(ts) for ts in df_tkn['timestamp'].values])
    tkn_coin_balances = df_tkn['tkn_balance'].values/(10**tkn_decimal)

    fig, (TKN_ax) = plt.subplots(nrows=1, sharex=False, sharey=False, figsize=(16, 3))
    TKN_ax.plot(dates, tkn_coin_balances, color = 'r',linestyle = 'dashdot', label=tkn_symbol) 
    if(plot_title): TKN_ax.set_title(f'Address: {tkn_addr}', size=20)
    TKN_ax.set_ylabel(f'{tkn_symbol} Balance', size=14)
    TKN_ax.set_xlabel('Date', size=14)
    plot_title = False